<a href="https://colab.research.google.com/github/vishwasjoshi2019/Stock_Market_Analysis/blob/main/Future_pricing_spread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Future Calender Spread Strategy**

Calculate the daily historic difference between the two contracts and generate a time series. Calculate the mean and standard deviation of the time series. Using the mean and standard deviation data we can estimate the range for the difference. A trading signal is triggered when the difference between the two contracts move to mean plus or minus 1 standard deviation and the trade is closed when the difference collapses to mean.


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Define the symbols for the current and next expiry futures contracts
current_contract_symbol = 'NQ=F'  # Change to the actual symbol
next_expiry_contract_symbol = 'NQZ23.CME'  # Change to the actual symbol

# Define the start and end dates for historical data
start_date = "2023-01-01"
end_date = "2023-12-31"

# Fetch historical data for both contracts
current_contract_data = yf.download(current_contract_symbol, start=start_date, end=end_date)
next_expiry_contract_data = yf.download(next_expiry_contract_symbol, start=start_date, end=end_date)

# Calculate the daily difference between the two contracts' closing prices
daily_difference = -current_contract_data['Adj Close'] +next_expiry_contract_data['Adj Close']

# Calculate the mean and standard deviation of the time series
mean_difference = daily_difference.mean()
std_deviation = daily_difference.std()

# Calculate the upper and lower trading signal thresholds (mean plus/minus 1 standard deviation)
upper_threshold = mean_difference + std_deviation
lower_threshold = mean_difference - std_deviation

# Create trading signals (1 for buy, -1 for sell)
trading_signals = np.where(daily_difference > upper_threshold, -1, np.where(daily_difference < lower_threshold, 1, 0))

# Create a time series for trading signals
signal_series = pd.Series(trading_signals, index=daily_difference.index)

# Create a Plotly graph to visualize the time series and trading signals
fig = go.Figure()

# Plot the daily difference
fig.add_trace(go.Scatter(x=daily_difference.index, y=daily_difference, mode='lines', name='Daily Difference'))

# Plot the upper and lower trading signal thresholds
fig.add_trace(go.Scatter(x=daily_difference.index, y=[upper_threshold] * len(daily_difference), mode='lines', name='Upper Threshold', line=dict(color='red')))
fig.add_trace(go.Scatter(x=daily_difference.index, y=[lower_threshold] * len(daily_difference), mode='lines', name='Lower Threshold', line=dict(color='green')))

# Plot buy (1) and sell (-1) signals
buy_signals = signal_series[signal_series == 1]
sell_signals = signal_series[signal_series == -1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals.values, mode='markers', name='Buy Signal', marker=dict(color='blue', size=6)))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals.values, mode='markers', name='Sell Signal', marker=dict(color='red', size=6)))

# Add titles and labels
fig.update_layout(
    title='NASDAQ Futures Trading Strategy',
    xaxis_title='Date',
    yaxis_title='Daily Difference',
)

# Show the plot
fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


#**SPOT VS Future theoritical value**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Define the start and end dates for the historical data
start_date = "2023-01-01"
end_date = "2023-12-31"

# Fetch historical SPX spot close prices
spx_data = yf.download('^GSPC', start=start_date, end=end_date)

# Define the risk-free interest rate (4.74% as per your request)
risk_free_rate = 0.0474

# Calculate the number of days to expiration (assuming 252 trading days in a year)
spx_data['DaysToExpiration'] = (spx_data.index[-1] - spx_data.index).days

# Calculate the theoretical value of SPX futures using the cost-of-carry model
spx_data['TheoreticalFutureValue'] = spx_data['Adj Close'] * np.exp(risk_free_rate * spx_data['DaysToExpiration'] / 252)

# Create a Plotly graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=spx_data.index, y=spx_data['Adj Close'], mode='lines', name='SPX Spot Close'))
fig.add_trace(go.Scatter(x=spx_data.index, y=spx_data['TheoreticalFutureValue'], mode='lines', name='Theoretical Future Value'))

# Add titles and labels
fig.update_layout(
    title='SPX Spot Close vs. Theoretical Future Value',
    xaxis_title='Date',
    yaxis_title='Price',
)

# Show the plot
fig.show()


[*********************100%%**********************]  1 of 1 completed


**# Future actual and theoritical**

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Define the start and end dates for the historical data
start_date = "2023-01-01"
end_date = "2023-12-31"

# Fetch historical SPX spot close prices
spx_spot_data = yf.download('^GSPC', start=start_date, end=end_date)

# Define the risk-free interest rate (4.74% as per your request)
risk_free_rate = 0.0474

# Define the SPX futures symbol for December (you may need to adjust this symbol)
spx_dec_futures_symbol = 'ESZ23.CME'

# Fetch historical SPX futures prices for December
spx_futures_data = yf.download(spx_dec_futures_symbol, start=start_date, end=end_date)

# Calculate the number of days to expiration for December futures (assuming 252 trading days in a year)
spx_futures_data['DaysToExpiration'] = (spx_futures_data.index[-1] - spx_futures_data.index).days

# Calculate the theoretical value of SPX futures using the cost-of-carry model
spx_futures_data['TheoreticalFutureValue'] = spx_spot_data['Adj Close'] * np.exp(risk_free_rate * spx_futures_data['DaysToExpiration'] / 252)

# Create a Plotly graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=spx_futures_data.index, y=spx_futures_data['Adj Close'], mode='lines', name='SPX Futures (Dec)'))
fig.add_trace(go.Scatter(x=spx_futures_data.index, y=spx_futures_data['TheoreticalFutureValue'], mode='lines', name='Theoretical Future Value'))

# Add titles and labels
fig.update_layout(
    title='SPX Futures (Dec) vs. Theoretical Future Value',
    xaxis_title='Date',
    yaxis_title='Price',
)

# Show the plot
fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
